In [26]:
%matplotlib qt
import csv
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage.util import random_noise
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA, KernelPCA
import math
import copy

import kpca

In [27]:
def pca(X):
    n = X.shape[0]
    Xcentered = X - np.mean(X,0)
    C = 1.0/n*Xcentered.T.dot(Xcentered)
    D, V = np.linalg.eigh(C)
    return (D, V)

def gaussianKernel(x, y, c):
	''' Returns K(x,y) where K denotes gaussian kernel '''
	return math.exp(-(np.linalg.norm(x-y)**2) / c)

def createK(data, kernelFunction, c):
	''' Returns K matrix containing inner products of the data using the kernel function 
	so that K_ij := (phi(x_i)*phi(x_j)) '''
	l = len(data)
	K = np.zeros((l,l))
	for col in range(l):
		for row in range(l):
			K[row][col] = kernelFunction(data[row],data[col], c)
	return K

def calcBetaK(alphaK, kernelFunction, data, x, c):
	''' Returns the projection of x onto the eigenvector V_k '''
	BetaK = 0
	for i,xi in enumerate(data):
		BetaK += alphaK[i]*kernelFunction(xi,x,c)
	return BetaK	
	
def centerK(K):
	''' Returns centered K matrix, see K. Murphy 14.43 '''
	l = len(K)
	l_ones = np.ones((l, l), dtype=int) / l
	Kcentered = K - np.dot(l_ones,K)-np.dot(K,l_ones)+np.dot(l_ones,np.dot(K,l_ones))	
	return Kcentered

def normAlpha(alpha, lambdas):
	''' Returns new alpha corresponding to normalized eigen vectors,
	so that lambda_k(a^k * a^k) = 1 '''
	for i,a in enumerate(alpha):
		a /= np.sqrt(lambdas[i])
	return alpha

def calcZold(alpha, data, x, kernelFunction, c,z0):
	''' Equation (10), returns pre-image z for single input datapoint x '''
	z = z0
	iters=0
	while iters <5:
		numerator = 0
		denom = 0
		for i, xi in enumerate(data):
			gammaI = calcGammaI(alpha, i, data, x, kernelFunction, c) * kernelFunction(z,xi,c)
			numerator += gammaI * xi
			denom += gammaI
		z = numerator/denom
		iters +=1
	return z

def calcZ(alpha, data, x, kernelFunction, c,z0):
	''' Equation (10), returns pre-image z for single input datapoint x '''
	z = z0
	iters=0
	maxIters = 1000
	# calculate beta (does not change with each iteration)
	beta = [calcBetaK(aK, kernelFunction, data, x, c) for aK in alpha]

	while iters < maxIters: # iterate until convergence
		numerator = 0
		denom = 0
		for i, xi in enumerate(data):
			#gammaI = calcGammaI(alpha, i, data, x, kernelFunction, c) * kernelFunction(z,xi,c)
			gammaI = calcGammaIOpt(alpha, i, beta) * kernelFunction(z,xi,c)
			numerator += gammaI * xi
			denom += gammaI
		if denom > 10**-12: #handling numerical instability
			newZ = numerator/denom
			if np.linalg.norm(z - newZ) < 10**-8:
				z = newZ
				break
			z = newZ
			iters += 1
		else:
			iters =0
			z=z0 + np.random.multivariate_normal(np.zeros(z0.size),np.identity(z0.size))
			numerator = 0
			denom = 0

	return z

def calcGammaI(alpha, i, data, x, kernelFunction, c):
	''' returns gamma_i = sum_{k=1}^n Beta_k * alpha_i^k '''
	gammaI = 0
	alphaI = alpha.T[i]
	for k, alphaKI in enumerate(alphaI):
		gammaI += calcBetaK(alpha[k], kernelFunction, data, x, c) * alphaKI
	return gammaI

def calcGammaIOpt(alpha, i, beta):
	''' returns gamma_i = sum_{k=1}^n beta_k * alpha_i^k '''
	gammaI = 0
	alphaI = alpha.T[i]
	for k, alphaKI in enumerate(alphaI):
		gammaI += beta[k] * alphaKI
	return gammaI

def kernelPCADeNoise(kernelFunction, c, components, dataTrain, dataTest):
	Data = dataTrain

	l = len(Data)

	# build K
	K = createK(Data, kernelFunction, c)

	# center K
	K = centerK(K)

	# find eigen vectors
	lLambda, alpha = np.linalg.eigh(K) # (3)
	lambdas = lLambda/l # /l with the notation from the paper (but not murphys) 
	# drop negative and 0 egienvalues and their vectors
	for i,l in enumerate(lambdas):
		if l > 10**(-8):
			lambdas = lambdas[i:]
			alpha = alpha[i:]
			break

	# use only the 4 larges eigen values with corresponding vectors
	lambdas=lambdas[-components:]
	alpha=alpha[-components:]

	# normalize alpha
	alpha = normAlpha(alpha, lambdas)

	Z =[]
	for i in range(len(dataTest)):
		Z.append(calcZ(alpha, Data, dataTest[i],kernelFunction,c,dataTest[i]))

	Z=np.array(Z)
	return Z

def add_white_noise(X):
    # add additive noise
    return X + np.random.normal(size=X.shape)/2

def add_sp_noise(X):
    # add salt and pepper noise
    Y = copy.deepcopy(X)
    Y[np.random.choice(range(256),size=40,replace=False)]=1
    return Y

## PCA 

In [28]:
# read usps dataset
usps = fetch_mldata('usps')
# size of the sample
n = 300
# find size amount of samples of number digit

#idx = np.random.choice(usps.data.shape[0],size=n, replace=False)

# randomly take n samples of every digit
idx = []
for digit in range(10):
    idx = idx + list(np.random.choice(np.where((usps.target-1) == digit)[0], size=n, replace=False))

Xtrain = usps.data[idx]

In [29]:
# perform pca
D, V = pca(Xtrain)

In [5]:
# Figure 2: plot of the eigenvectors of C (PCA) 

fig = plt.figure()

for i in range(8):  
    fig.add_subplot(1,8,i+1)
    plt.imshow(V.T[-(2**i)].reshape((16,16)), cmap=plt.cm.Greys, interpolation='none')
    plt.axis('off') 

plt.show()

In [6]:
# Figure 3: plot the reconstructions using PCA

fig = plt.figure()

X = Xtrain[np.random.randint(Xtrain.shape[0])]

for i in range(20):  
    fig.add_subplot(1,21,i+1)
    Xapprox = X.dot(V[:,-i-1:]).dot(V[:,-i-1:].T)
    plt.imshow(Xapprox.reshape((16,16)), cmap=plt.cm.Greys, interpolation='none')
    plt.axis('off') 
    error = np.linalg.norm(X-Xapprox)
    plt.title(round(error,2))

fig.add_subplot(1,21,21)
plt.imshow(X.reshape((16,16)), cmap=plt.cm.Greys, interpolation='none')
plt.axis('off') 


plt.show()

In [33]:
# Figure 4: Denoising using PCA
fig = plt.figure()

for i in range(10):
    # plot 1 sample from every digit class
    Xsample = Xtrain[i*n]
    
    fig.add_subplot(7,20,i+1)    
    plt.imshow(Xsample.reshape((16,16)), cmap=plt.cm.Greys, interpolation='none')
    plt.axis('off') 
    
    fig.add_subplot(7,20,10+i+1)    
    plt.imshow(Xsample.reshape((16,16)), cmap=plt.cm.Greys, interpolation='none')
    plt.axis('off') 
    
    # plot noisy version of it
    Xwn = add_white_noise(Xsample)
    Xsp = add_sp_noise(Xsample)
    
    fig.add_subplot(7,20,20+i+1)    
    plt.imshow(Xwn.reshape((16,16)), cmap=plt.cm.Greys, interpolation='none')
    plt.axis('off') 
    
    fig.add_subplot(7,20,30+i+1)    
    plt.imshow(Xsp.reshape((16,16)), cmap=plt.cm.Greys, interpolation='none')
    plt.axis('off') 
    
    for j in range(5):
        fig.add_subplot(7,20,40+j*20+i+1)
        Xapprox = Xwn.dot(V[:,-4**j:]).dot(V[:,-4**j:].T)
        plt.imshow(Xapprox.reshape((16,16)), cmap=plt.cm.Greys, interpolation='none')
        plt.axis('off') 
        
        fig.add_subplot(7,20,50+j*20+i+1)
        Xapprox = Xsp.dot(V[:,-4**j:]).dot(V[:,-4**j:].T)
        plt.imshow(Xapprox.reshape((16,16)), cmap=plt.cm.Greys, interpolation='none')
        plt.axis('off') 


plt.show()

## KPCA

In [131]:
# read usps dataset
usps = fetch_mldata('usps')
# size of the sample
n = 500
# find size amount of samples of number digit
digit = 5
idx = np.random.choice(usps.data.shape[0],size=n, replace=False)
#idx = np.random.choice(np.where((usps.target-1) == digit)[0], size=n, replace=False)

# declare training data
Xtrain = usps.data[idx]

In [136]:
Data = Xtrain

l = len(Data)
c = 0.5
components = 200

# build K
K = createK(Data, gaussianKernel, c)

# center K
K = centerK(K)

# find eigen vectors
lLambda, alpha = np.linalg.eigh(K) # (3)
lambdas = lLambda/l # /l with the notation from the paper (but not murphys) 
# drop negative and 0 egienvalues and their vectors
for i,l in enumerate(lambdas):
	if l > 10**(-8):
		lambdas = lambdas[i:]
		alpha = alpha[i:]
		break

# use only the 4 larges eigen values with corresponding vectors
lambdas=lambdas[-components:]
alpha=alpha[-components:]

# normalize alpha
alpha = normAlpha(alpha, lambdas)

In [139]:

Z = calcZ(alpha, Xtrain, Xtrain[1],gaussianKernel,c,Xtrain[i])


KeyboardInterrupt: 